<h1>BERT</h1>

<li>BERT는 transformer encoder 아키텍쳐중 하나로, NLP에서 아주 성공적으로 사용되고 있다.<br>
<li>이 아키텍쳐는 자연어를 vector space에 올려놓고 deep learning을 적용한다.<br>
<li>BERT failmily의 모들들은 transformer 인커더 아키텍쳐를 사용해서 full context를 유지 한 채로, 각 text 인풋 토큰을 양방향으로 processing한다.<br>
<li>BERT 모델들은 주로 미리 커다란 corpus of text를 사용해서 <b>미리 train 시켜놓고</b> 각 모델의 <b>사용용도에 맞춰 fine-tune해서 사용한다.

In [2]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # this is for AdamW optimizer -- 스페셜한 optimizer인가보네?
import matplotlib.pyplot as plt
tf.get_logger().setLevel('ERROR')

 이 노트북에서는 Sentiment analysis를 시도해 본다.<br>
 Movie review 받아서 Positive인지 Negative인지 알아본다.

In [3]:
# IMDB 데이터를 다운받자.
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

dataset = tf.keras.utils.get_file('aclImdb_v1.tar.gz', url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

train_dir = os.path.join(dataset_dir, 'train')

# remove unused folders to make it easier to load the data
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

84131840/84125825 [==============================] - 39s 0us/step


이제 text_dataset_from_directory를 사용해서 tf.data.Dataset obj를 만들것이다.<br>
또, 데이터는 train / test만 있고 validation set가 없으므로 train세트를 8:2로 나눠서 validation set을 만들자.

In [24]:
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42

# batch 사이즈나 shuffle을 dataset단계에서 해결한다.
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

class_names = raw_train_ds.class_names
train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE) # Dataset는 모든 데이터를 한번에 불러오지 않는다.
# 한 batch씩 읽고 train 할 동안에 다음 batch를 미리 load 시켜놓자.

val_ds = tf.keras.preprocessing.text_dataset_from_directory(
'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

test_ds = tf.keras.preprocessing.text_dataset_from_directory('aclImdb/test', batch_size=batch_size)  # tets세트는 섞을 이유가 없다.
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


In [78]:
for text_batch, label_batch in train_ds.take(1):
    for i in range(3):
        print(f"review: {text_batch.numpy()[i]}")
        label=label_batch.numpy()[i]
        print(f'Label : {label} ({class_names[label]})')        

review: b'This cowardly and offensive film had me intrigued to begin with. The characters are the familiar dispossessed young males frequently to be seen hanging around bored in a sea side town. Robert is an outsider but he has his music which could have been his soul. Instead Clay makes Robert into a freak who embarks on a journey into cannabis and ecstasy and getting in with the wrong crowd. Clay seems to believe in "reefer madness" and Robert ends the film as a homicidal rapist. One wonders how much experience of real life this young director has. No one can save poor Robert. Clay leaves us with the message that young British men are out of control. A very unsubtle link is made to the Iraqi insurgents; during the needlessly graphic rape we are subjected to explosions and images of war. The film shows male peer group extremism pushed to it\'s limits. The young bombers in London draw a parallel with Clay\'s hateful depiction of modern male. Clay implies that men simply cannot help the

In [63]:
for text_batch, label_batch in train_ds.take(1):
    print(label_batch.numpy())

[0 1 1 1 1 0 1 0 0 0 0 1 1 1 0 1 1 0 1 1 0 0 0 1 0 0 1 1 0 0 0 0]


아까 말했던 대로, BERT 모델은 미리 train되어 있는 것을 불러와서 내 목적에 맞게 fine tune해서 쓸꺼야.<br>
Google에서 만든 여러 모델이 있는데, 
모델 불러오자.